In [1]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152
import numpy as np
# import matplotlib
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model1 = resnet18(pretrained=True).to(device)
model2 = resnet34(pretrained=True).to(device)
model3 = resnet50(pretrained=True).to(device)
model4 = resnet101(pretrained=True).to(device)
model5 = resnet152(pretrained=True).to(device)

def get_gradcam_saliency_maps(x, y, model):
  with GradCAM(model=model, target_layers=[model.layer4[-1]], use_cuda=torch.cuda.is_available()) as cam:
      targets = [ClassifierOutputTarget(y)]

      # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
      grayscale_cam = cam(input_tensor=x, targets=targets)

      # TODO: Do we want to use eigen_smooth=True in the above function to remove some of the noise??

  return grayscale_cam

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.76 GiB total capacity; 98.95 MiB already allocated; 14.75 MiB free; 110.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def compare_saliency_maps(sm1, sm2):
    return np.abs(sm1 - sm2).sum()

In [7]:
train = datasets.CIFAR10(root = '/home/jupyter/cs762/data', train=True,  transform=transforms.Compose([transforms.ToTensor(), transforms.Resize([224, 224])]), download=True)
data_loader = torch.utils.data.DataLoader(train, batch_size=1,
                                                 shuffle=True, num_workers=1)

models_list = [model1, model2, model3, model4, model5]

model_heatmap = np.array([[0 for i in range(len(models_list))] for j in range(len(models_list))], dtype=float)

# about 20s per batch
for batch in data_loader:  # Get batch
    images, labels = batch  # Unpack the batch into images and labels
    # labels = labels - labels
    # images, labels = images.to(device), labels.to(device)

    # look at all labels
    for label in range(10):
        labels = torch.tensor([label])
        images, labels = images.to(device), labels.to(device)

        for model_index1 in range(len(models_list)):
            for model_index2 in range(len(models_list)):
                sm1 = get_gradcam_saliency_maps(images, labels, models_list[model_index1])
                sm2 = get_gradcam_saliency_maps(images, labels, models_list[model_index2])
                model_heatmap[model_index1, model_index2] += compare_saliency_maps(sm1, sm2)/len(data_loader.dataset)
        
    plt.imshow(model_heatmap)
    plt.title("model comparison heatmap")
    plt.colorbar()
    plt.show()

170499072it [00:02, 59343094.58it/s]                               


Extracting /home/jupyter/cs762/data/cifar-10-python.tar.gz to /home/jupyter/cs762/data


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED

In [ ]:
plt.imshow(model_heatmap)
plt.title("model comparison heatmap")
plt.colorbar()
plt.show()